# GeoJSON polígons
Codi per generar els GeoJSON de les vegueries i les comarques rellevants per al repte.

In [1]:
import json
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
## Nombre de comarques que surten al repte

TOTAL_COMARQUES = ['Alt Camp', 'Noguera', 'Baix Camp', 'Barcelonès', 'Priorat', 'Baix Ebre', 'Segrià', "Ribera d'Ebre", "Pla d'Urgell", 'Alt Penedès', 'Gironès', 'Vallès Occidental', 'Terra Alta', 'Osona', 'Baix Penedès', 'Montsià', 'Pallars Jussà', 'Baix Llobregat', 'Bages', "Pla de l'Estany", 'Anoia', 'Garrigues', 'Alt Empordà', 'Alt Urgell', 'Vallès Oriental', 'Segarra', 'Moianès', 'Vallespir', 'Ripollès', 'Berguedà', 'Maresme', 'Cerdanya', 'Alta Cerdanya', 'Tarragonès', 'Garrotxa', 'Alta Ribagorça', 'Solsonès', "Aran", 'Conflent', 'Andorra', 'Garraf', 'Baix Empordà', 'Conca de Barberà', 'Rosselló', 'Urgell', 'Selva', 'Pallars Sobirà', 'Capcir']

## Importació del GeoJSON de municipis
S'eliminen les columnes que no interessen.

In [ ]:
df = gpd.read_file("municipis_Cat_And_CN_Franja.geojson").drop(columns=["extrude", "cartodb_id", "visibility", "codi", "municipi", "OGR", "OGR_STYLE", "tessellate"])

df.loc[df["territori"] == "CAT", "territori"] = 1
df.loc[df["territori"] == "AND", "territori"] = 2
df.loc[df["territori"] == "NORD", "territori"] = 3
df.loc[df["territori"] == "FRANJA", "territori"] = 4
df

## GeoJSON comarques
S'importa un CSV amb el nom de les comarques i el seu codi identificatiu.

In [ ]:
comarques_extra = pd.DataFrame({"Codi": [43, 44, 45, 46, 47, 48],
                                "comarca": ["Andorra", "Capcir", "Conflent", "Rosselló", "Vallespir", "Alta Cerdanya"]})
comarques_extra

In [ ]:
comarques_id = pd.read_csv("comarques_codi.csv").drop(columns=["Unnamed: 3"])
comarques_id = comarques_id[(comarques_id["Nivell"] == "Comarca") | (comarques_id["Nivell"] == "Ent. terr. sing.")].drop(columns=["Nivell"]).rename(columns={"Nom": "comarca"}).reset_index().drop(columns="index").append(comarques_extra, ignore_index=True)
comarques_id.rename(columns={"Codi": "id"}).set_index("id")

Es fusionen els polígons municipals per obtenir els polígons de comarca. Després, es filtren les comarques segons si es troben a `TOTAL_COMARQUES` i se'ls hi afegeix la id de comarca.

In [ ]:
comarques = df.dissolve(by=["comarca", "territori"]).reset_index().rename(columns={"territori": 'territori_id'})
comarques = pd.merge(comarques, comarques_id, on="comarca", how="outer").rename(columns={"Codi": "comarca_id"})
comarques = comarques.drop([56])
comarques["comarca_id"] = comarques["comarca_id"].fillna(0).astype("int")
comarques["territori_id"] = comarques["territori_id"].astype("int")
conditions = [(comarques["territori_id"] == 1), (comarques["territori_id"] == 2), (comarques["territori_id"] == 4), (comarques["territori_id"] == 3)]
values = ["Catalunya", "Andorra", "La Franja", "Catalunya Nord"]
comarques["territori"] = np.select(conditions, values)
comarques = comarques[comarques["comarca"].isin(TOTAL_COMARQUES)]

comarques.loc[22, "comarca"] = "Alta Cerdanya"
comarques.loc[22, "comarca_id"] = 48

comarques.loc[6, "vegueria"] = "Andorra"

comarques

S'exporta en format GeoJSON.

In [ ]:
with open("comarques_parsed.geojson", "w") as f:
    f.write(comarques.to_json(drop_id=True))

## GeoJSON vegueries
Es duu a terme el mateix procés que amb les comarques però agrupant per vegueries. Es crea un id aleatori del 0 al 13 i s'exporta com a GeoJSON.

In [ ]:
vegueries = df.dissolve(by="vegueria").reset_index().sort_values(by=["territori", "vegueria"]).reset_index().rename(
    columns={"index": "vegueria_id"}).drop(columns=["comarca"])
vegueries

In [ ]:
vegueries_id = vegueries[["vegueria_id", "vegueria"]].rename(columns={"vegueria_id": "id"}).set_index("id")
vegueries_id

In [ ]:
with open("vegueries.geojson", "w") as f:
    f.write(vegueries.to_json(drop_id=True))

In [ ]:
## S'actualitza el GeoJSON de comarques per incloure l'id de les vegueries.

comarques = gpd.read_file("comarques_parsed.geojson")
comarques = pd.merge(comarques, vegueries, on=["vegueria"], how="outer").drop(columns=["geometry_y", "territori_y"]).rename(columns={"territori_x": "territori", "geometry_x": "geometry"})
comarques["vegueria_id"] = comarques["vegueria_id"].fillna(14).astype(int)
with open("comarques_parsed.geojson", "w") as f:
    f.write(comarques.to_json(drop_id=True))

In [ ]:
comarques_vegueries = comarques[["comarca", "comarca_id", "vegueria", "vegueria_id"]].sort_values("vegueria").groupby(["vegueria", "vegueria_id"]).agg({"comarca": lambda x: list(x), "comarca_id": lambda x: list(x)}).reset_index()
comarques_vegueries

In [ ]:
comarques_vegueries_json = []
for i, row in comarques_vegueries.iterrows():
    vegueria = {
        "name": row["vegueria"],
        "id": row["vegueria_id"],
        "children": [{"name": comarca, "id": row["comarca_id"][j]} for j, comarca in enumerate(row["comarca"])]
    }
    comarques_vegueries_json.append(vegueria)

with open("comarques_vegueries.json", "w") as f:
    f.write(json.dumps(comarques_vegueries_json, ensure_ascii=False, indent=4))
comarques_vegueries_json

## GeoJSON Territoris

In [ ]:
comarques[["territori_id", "territori"]]

In [ ]:
territori = comarques[["territori", "territori_id"]].drop_duplicates().sort_values("territori_id", ascending=True).set_index("territori_id")
territori

## 100 cims topo
S'afegeix la informació topogràfica (territori, vegueria, comarca) al JSON de 100 cims.

In [ ]:
cent_cims_df = pd.read_json("100cims.json")

comarques_cent_cims = cent_cims_df["comarca"].to_list()

cent_cims_df["territory"] = [[comarques[comarques["comarca"] == item]["territori"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]
cent_cims_df["territory_id"] = [[comarques[comarques["comarca"] == item]["territori_id"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]
cent_cims_df["comarca_id"] = [[comarques[comarques["comarca"] == item]["comarca_id"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]
cent_cims_df["vegueria"] = [[comarques[comarques["comarca"] == item]["vegueria"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]
cent_cims_df["vegueria"] = [[comarques[comarques["comarca"] == item]["vegueria"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]
cent_cims_df["vegueria_id"] = [[comarques[comarques["comarca"] == item]["vegueria_id"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]

In [ ]:
cent_cims_df.to_json("100cims_2.json", orient="records")

In [ ]:
comarques_cent_cims

Es genera el JSON amb l'estructura definitiva (que després s'anirà ampliant).

In [ ]:
with open("100cims_2.json") as f:
    cent_cims = json.load(f)
cent_cims2 = []

for cim in cent_cims:
    cim2 = {
        "name": cim["nom"],
        "id": cim["id"],
        "coords": {
            "latlng": {
                "lat": cim["lat"],
                "lng": cim["lng"]
            },
            "utm": {
                "x": cim["utm_x"],
                "y": cim["utm_y"]
            }
        },
        "topography": {
            "territory": cim["territory_id"],
            "vegueria": cim["vegueria_id"],
            "comarca": cim["comarca_id"]
        },
        "essential": cim["essencial"],
        "url": {
            "feec": cim["url"],
            "wikidata": cim["wikidata"]
        },
        "images": [] if cim["image"] is None else [cim["image"]],
        "height": cim["altitud"]
    }
    cent_cims2.append(cim2)

with open("100cims_2.json", "w") as f:
    f.write(json.dumps(cent_cims2, ensure_ascii=False, indent=4))

In [23]:
territori = comarques[["territori", "territori_id"]].drop_duplicates().sort_values("territori_id", ascending=True).set_index("territori_id")
territori

,territori
territori_id,
1,Catalunya
2,Andorra
3,Catalunya Nord
4,La Franja


## 100 cims topo
S'afegeix la informació topogràfica (territori, vegueria, comarca) al JSON de 100 cims.

In [29]:
cent_cims_df = pd.read_json("100cims.json")

comarques_cent_cims = cent_cims_df["comarca"].to_list()

cent_cims_df["territory"] = [[comarques[comarques["comarca"] == item]["territori"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]
cent_cims_df["territory_id"] = [[comarques[comarques["comarca"] == item]["territori_id"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]
cent_cims_df["comarca_id"] = [[comarques[comarques["comarca"] == item]["comarca_id"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]
cent_cims_df["vegueria"] = [[comarques[comarques["comarca"] == item]["vegueria"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]
cent_cims_df["vegueria"] = [[comarques[comarques["comarca"] == item]["vegueria"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]
cent_cims_df["vegueria_id"] = [[comarques[comarques["comarca"] == item]["vegueria_id"].iloc[0] for item in comarca] for comarca in comarques_cent_cims]

KeyError: 'vegueria_id'

In [204]:
cent_cims_df.to_json("100cims_2.json", orient="records")

In [141]:
comarques_cent_cims

[['Alt Camp'],
 ['Conflent', 'Vallespir'],
 ['Rosselló'],
 ['Alta Cerdanya'],
 ['Conflent'],
 ['Alta Cerdanya'],
 ['Capcir', 'Alta Cerdanya'],
 ['Conflent'],
 ['Alta Cerdanya', 'Conflent'],
 ['Andorra'],
 ['Andorra'],
 ['Andorra'],
 ['Vallès Oriental'],
 ['Andorra', 'Alta Cerdanya'],
 ['Vallès Occidental'],
 ['Vallès Oriental'],
 ['Andorra'],
 ['Alta Cerdanya', 'Ripollès'],
 ['Vallès Occidental'],
 ['Solsonès'],
 ['Vallès Occidental'],
 ['Selva', 'Vallès Oriental'],
 ["Val d'Aran"],
 ["Val d'Aran"],
 ["Val d'Aran"],
 ["Val d'Aran"],
 ['Segrià'],
 ['Urgell'],
 ['Vallès Occidental'],
 ['Solsonès'],
 ['Terra Alta'],
 ['Selva'],
 ['Terra Alta'],
 ['Ripollès'],
 ['Segarra'],
 ["Val d'Aran"],
 ["Val d'Aran"],
 ['Selva'],
 ['Ripollès'],
 ['Conflent', 'Ripollès'],
 ['Ripollès'],
 ['Ripollès', 'Vallespir'],
 ["Pla d'Urgell"],
 ["Ribera d'Ebre"],
 ["Ribera d'Ebre"],
 ["Ribera d'Ebre"],
 ['Priorat'],
 ["Pla de l'Estany"],
 ['Pallars Sobirà'],
 ['Pallars Sobirà', "Val d'Aran"],
 ['Pallars Sobirà']

Es genera el JSON amb l'estructura definitiva (que després s'anirà ampliant).

In [209]:
with open("100cims_2.json") as f:
    cent_cims = json.load(f)
cent_cims2 = []

for cim in cent_cims:
    cim2 = {
        "name": cim["nom"],
        "id": cim["id"],
        "coords": {
            "latlng": {
                "lat": cim["lat"],
                "lng": cim["lng"]
            },
            "utm": {
                "x": cim["utm_x"],
                "y": cim["utm_y"]
            }
        },
        "topography": {
            "territory": cim["territory_id"],
            "vegueria": cim["vegueria_id"],
            "comarca": cim["comarca_id"]
        },
        "essential": cim["essencial"],
        "url": {
            "feec": cim["url"],
            "wikidata": cim["wikidata"]
        },
        "images": [] if cim["image"] is None else [cim["image"]],
        "height": cim["altitud"]
    }
    cent_cims2.append(cim2)

with open("100cims_2.json", "w") as f:
    f.write(json.dumps(cent_cims2, ensure_ascii=False, indent=4))

In [39]:
with open("centcims.json", encoding="utf-8") as f:
    cent_cims = json.load(f)

for cim in cent_cims:
    cim["topography"]["territory"] = [territory["id"] for territory in cim["topography"]["territory"]]
    cim["topography"]["comarca"] = [comarca["id"] for comarca in cim["topography"]["comarca"]]
    cim["topography"]["vegueria"] = [vegueria["id"] for vegueria in cim["topography"]["vegueria"]]

with open("100cims_2.json", "w") as f:
    f.write(json.dumps(cent_cims, ensure_ascii=False, indent=4))
cent_cims

[{'name': 'El Cogulló de Cabra',
  'id': 639,
  'coords': {'latlng': {'lat': 41.4290355033, 'lng': 1.3057579489},
   'utm': {'x': 358437, 'y': 4587771}},
  'topography': {'territory': ['CAT'], 'vegueria': [3], 'comarca': [1]},
  'essential': True,
  'url': {'feec': 'https://www.feec.cat/activitats/100-cims/cim/el-cogullo-de-cabra/',
   'wikidata': 'http://www.wikidata.org/entity/Q17481698',
   'wikipedia': 'https://ca.wikipedia.org/wiki/El_Cogull%C3%B3_(el_Pont_d%27Armentera)'},
  'images': ['https://upload.wikimedia.org/wikipedia/commons/b/bd/El_Cogull%C3%B3_des_de_Montblanc_P1250216.jpg'],
  'height': 881,
  'summary': "El Cogulló és una muntanya de 881 metres que es troba al municipi del Pont d'Armentera, a la comarca de l'Alt Camp.",
  'ascensions': {'total': 727,
   'monthly': [94, 98, 64, 98, 53, 36, 21, 29, 43, 71, 51, 69]}},
 {'name': 'Puig de Tretzevents',
  'id': 637,
  'coords': {'latlng': {'lat': 42.492101894, 'lng': 2.4683643295},
   'utm': {'x': 456312, 'y': 4704553}},
